In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
df = pd.read_csv('datasets_hotels.csv')
df.head()

,id_hotels,name,rating,number_of_ratings,rating_title,url,price,room_type,adress,location,Distance,Content,price_for,People numbers
0,0,Lan Vy Hotel,9.7,37 reviews,Fabulous,https://www.booking.com/hotel/vn/lan-vy-can-th...,23,Superior Double Room,"97D Pham Ngoc Thach, Cần Thơ, Việt Nam",Ninh Kiều,1.3 km from centre,Lan Vy Hotel is located in Can Tho. Only 1.3 k...,"1 night, 1 adult",1 double bed
1,1,Amazon 1,7.9,301 reviews,Good,https://www.booking.com/hotel/vn/amazon-1.en-g...,5,Bed in 4-Bed Dormitory Room,"88/24 Lê Lai, An Phú, Ninh Kiều, Thành phố Cần...",Ninh Kiều,0.4 km from centre\n,Situated in Can Tho and with Vincom Plaza Xuan...,"1 night, 1 adult",1 bunk bed
2,2,VẠN LỘC HOTEL,8.4,342 reviews,Very good,https://www.booking.com/hotel/vn/van-loc.en-gb...,15,Standard King Room,"76 Dong Khoi Street, Cần Thơ, Việt Nam",Ninh Kiều,0.4 km from centre\r,"Set in Can Tho, 800 metres from Ninh Kieu Pier...","1 night, 1 adult",1 large double bed
3,3,CALI HOTEL,7.7,285 reviews,Good,https://www.booking.com/hotel/vn/cali-can-tho1...,14,Standard King Room,"46 Lương Định Của, Phường Cái Khế, , Cái Khế, ...",Ninh Kiều,1.3 km from centre\r,"Located in Can Tho, within 1.1 km of Vincom Pl...","1 night, 1 adult",1 large double bed
4,4,Mekong Rose Hotel,8.3,81 reviews,Very good,https://www.booking.com/hotel/vn/mekong-rose-n...,11,Economy Double Room,"59-61 Ly Hong Thanh, P. Cai Khe, Q. Ninh Kieu,...",Ninh Kiều,1.4 km from centre\r,"Featuring a terrace, Mekong Rose Hotel is loca...","1 night, 1 adult",1 double bed


In [ ]:
# = df['rating']*1000

In [ ]:
#df['rating1']=df['rating1'].to_numeric(s, downcast='integer')

In [ ]:
#def sentiment(rating_title):
  #if rating in range(0,5.0):
    #return 'Dưới 1 triệu'
  #elif rating in range(5.1,7.5):
    #return '1 triệu - 2 triệu'
  #else:
    #return 'Trên 2 triệu'

In [ ]:
#df['rating1'] = df['rating'].apply(sentiment)
#df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_hotels          461 non-null    int64  
 1   name               461 non-null    object 
 2   rating             461 non-null    float64
 3   number_of_ratings  461 non-null    object 
 4   rating_title       266 non-null    object 
 5   url                461 non-null    object 
 6   price              460 non-null    float64
 7   room_type          461 non-null    object 
 8   adress             461 non-null    object 
 9   location           460 non-null    object 
 10  Distance           461 non-null    object 
 11  Content            461 non-null    object 
 12  price_for          461 non-null    object 
 13  People numbers     461 non-null    object 
dtypes: float64(2), int64(1), object(11)
memory usage: 50.5+ KB


In [4]:
# De xuất theo khoảng cách
tfidf = TfidfVectorizer()
df['Distance'] = df['Distance'].fillna('')
#Xây dựng ma trận TF-IDF bắt buộc bằng cách áp dụng phương thức fit_transform trên tính năng tổng quan
overview_matrix = tfidf.fit_transform(df['Distance'])
#Xuất ra hình dạng của tfidf_matrix
overview_matrix.shape

(461, 4)

In [5]:
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [6]:
#hotel index mapping
mapping = pd.Series(df.index,index = df['Distance'])
mapping

Distance
1.3 km from centre        0
0.4 km from centre\n      1
0.4 km from centre\r      2
1.3 km from centre\r      3
1.4 km from centre\r      4
                       ... 
3.8 km from centre\r    456
0.6 km from centre\r    457
3.2 km from centre\r    458
1.7 km from centre\r    459
8.1 km from centre      460
Length: 461, dtype: int64

In [11]:
def recommend_hotel_based_on_plot(hotel_input):
  hotel_index = mapping[hotel_input]
  #nhận các giá trị tương tự với khách sạn khác
  #điểm tương tự là danh sách chỉ số và ma trận tương tự
  similarity_score = list(enumerate(similarity_matrix[hotel_index]))
  #sắp xếp theo thứ tự giảm dần điểm tương đồng của khách sạn được nhập với tất cả các khách sạn khác
  similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
  # Nhận điểm của 5 khách sạn giống nhau nhất. Bỏ qua khách sạn đầu tiên.
  similarity_score = similarity_score[1:6]
  #trả về tên khách sạn bằng cách sử dụng chuỗi ánh xạ
  hotel_indices = [i[0] for i in similarity_score]
  return (df[['name','price']].iloc[hotel_indices])

In [12]:
recommend_hotel_based_on_plot('1.2 km from centre')

,name,price
13,Song Ngư Pisces Homestay,25
12,Linh Phuong 3 Hotel,35
11,Linh Phuong 2 Hotel,27
10,Mekong Delta Ricefield Lodge,42
9,Anh Dao Mekong 2 Hotel,30


In [13]:
df['price'] = df['price'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_hotels          461 non-null    int64  
 1   name               461 non-null    object 
 2   rating             461 non-null    float64
 3   number_of_ratings  461 non-null    object 
 4   rating_title       266 non-null    object 
 5   url                461 non-null    object 
 6   price              461 non-null    int64  
 7   room_type          461 non-null    object 
 8   adress             461 non-null    object 
 9   location           460 non-null    object 
 10  Distance           461 non-null    object 
 11  Content            461 non-null    object 
 12  price_for          461 non-null    object 
 13  People numbers     461 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 50.5+ KB


In [ ]:
#df['price'] = df['price'].strip ( 'US$' )
#df.head()

In [14]:
def sentiment(price):
  if price in range(0,10):
    return 'Giá Thấp'
  elif price in range(11,20):
    return 'Giá Trung Bình'
  else:
    return 'Giá Cao' 

df['price_list'] = df['price'].apply(sentiment)
df.head()

,id_hotels,name,rating,number_of_ratings,rating_title,url,price,room_type,adress,location,Distance,Content,price_for,People numbers,price_list
0,0,Lan Vy Hotel,9.7,37 reviews,Fabulous,https://www.booking.com/hotel/vn/lan-vy-can-th...,23,Superior Double Room,"97D Pham Ngoc Thach, Cần Thơ, Việt Nam",Ninh Kiều,1.3 km from centre,Lan Vy Hotel is located in Can Tho. Only 1.3 k...,"1 night, 1 adult",1 double bed,Giá Cao
1,1,Amazon 1,7.9,301 reviews,Good,https://www.booking.com/hotel/vn/amazon-1.en-g...,5,Bed in 4-Bed Dormitory Room,"88/24 Lê Lai, An Phú, Ninh Kiều, Thành phố Cần...",Ninh Kiều,0.4 km from centre\n,Situated in Can Tho and with Vincom Plaza Xuan...,"1 night, 1 adult",1 bunk bed,Giá Thấp
2,2,VẠN LỘC HOTEL,8.4,342 reviews,Very good,https://www.booking.com/hotel/vn/van-loc.en-gb...,15,Standard King Room,"76 Dong Khoi Street, Cần Thơ, Việt Nam",Ninh Kiều,0.4 km from centre\r,"Set in Can Tho, 800 metres from Ninh Kieu Pier...","1 night, 1 adult",1 large double bed,Giá Trung Bình
3,3,CALI HOTEL,7.7,285 reviews,Good,https://www.booking.com/hotel/vn/cali-can-tho1...,14,Standard King Room,"46 Lương Định Của, Phường Cái Khế, , Cái Khế, ...",Ninh Kiều,1.3 km from centre\r,"Located in Can Tho, within 1.1 km of Vincom Pl...","1 night, 1 adult",1 large double bed,Giá Trung Bình
4,4,Mekong Rose Hotel,8.3,81 reviews,Very good,https://www.booking.com/hotel/vn/mekong-rose-n...,11,Economy Double Room,"59-61 Ly Hong Thanh, P. Cai Khe, Q. Ninh Kieu,...",Ninh Kiều,1.4 km from centre\r,"Featuring a terrace, Mekong Rose Hotel is loca...","1 night, 1 adult",1 double bed,Giá Trung Bình


In [15]:
#theo gia
tfidf = TfidfVectorizer()
df['price_list'] = df['price_list'].fillna('')

overview_matrix = tfidf.fit_transform(df['price_list'])

overview_matrix.shape

similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

mapping = pd.Series(df.index,index = df['price_list'])


def recommend_hotel_based_on_plot(hotel_input):
  hotel_index = mapping[hotel_input]
  similarity_score = list(enumerate(similarity_matrix[hotel_index]))
  similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
  similarity_score = similarity_score[0:5]
  hotel_indices = [i[0] for i in similarity_score]
  return (df['name'].iloc[hotel_indices])

In [16]:
recommend_hotel_based_on_plot('Giá Thấp')

45                         08 Homestay
44                      BangBang Motel
43    Little Dalat Coffee and Homestay
42                       Olympus Hotel
41              Winter Spring Homestay
Name: name, dtype: object